In [2]:
# all the imports (required libs: numpy, matplotlib, scipy, libros, opensimplex (only when using perlin noise))
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
import IPython.display as ipd
import librosa
from random import shuffle
import opensimplex as ospx


# midpoint circle algorithm / algorithm to create rasterized circles (used to generate the circle search pattern for the wave terrain)
def circleDraw(radius):

    radius = radius - 1 

    def mirror_points_8(x, y):
        return [( x,  y),
            ( y,  x),
            (-x,  y),
            (-y,  x),
            ( x, -y),
            ( y, -x),
            (-x, -y),
            (-y, -x)]

    points = []
    x = 0
    y = radius
    # F_M is a float.
    F_M = 5 / 4 - radius
    points.extend(mirror_points_8(x, y))

    while x < y:

        if F_M < 0:
            F_M += 2.0 * x + 3.0
        else:
            F_M += 2.0 * (x - y) + 5.0
            y -= 1
        x += 1
        points.extend(mirror_points_8(x, y))

    return points 


# complicated and inefficient but functional algorithm, rasterizing a straight line while keeping the amount of line-squares constant for all possible line-angels 
def lineDraw(xStart,yStart,xEnd,yEnd,lineLength):

    if(xEnd == xStart):
        points = []

        if(yStart < yEnd):
            for i in range(lineLength):
                points.append([xStart, yStart + i])
        else:
            for i in range(lineLength):
                points.append([xStart, yStart - i])        

    else:

        slope = (yEnd - yStart) / (xEnd - xStart)
        lineSections = np.linspace(xStart,xEnd,lineLength)
        lineSections = np.delete(lineSections, 0)
        points = [[xStart,yStart]]

        for floatX in lineSections:

            floatY = yStart + (slope * (floatX - xStart)) 
            x = round(floatX)
            y = round(floatY)

            if((points[-1][0] == x) and (points[-1][1] == y)):

                xError = floatX % 1
                yError = floatY % 1
                xCorrection = -1
                yCorrection = -1

                if(floatX >=  x):
                    xError = 1 - xError
                    xCorrection = 1

                if(floatY >= y):
                    yError = 1 - yError
                    yCorrection = -1

                if(xError < yError):
                    x = x + xCorrection

                if(yError <= xError):
                    y = y + yCorrection   

            points.append([x,y])            

    return points    

# Algorithms for creating different wavetables

In [3]:
# generate random arrary that is then interpolated for smoother audio

randomSize = 700
interpSize = 1000

# genarate random array to interpolate (uniform, normal)
# baseArray = np.random.default_rng().uniform(low=-1.0, high=1.0, size=(randomSize,randomSize))
baseArray = np.random.default_rng().normal(loc=0.0, scale=1.0, size=(randomSize,randomSize))

x = np.linspace(0,randomSize,randomSize)
y = np.linspace(0,randomSize,randomSize)

f = interp2d(x, y, baseArray, kind='linear')

nX = np.linspace(0,randomSize,interpSize)
nY = np.linspace(0,randomSize,interpSize)

interpArray = f(nX,nY)

In [10]:
# perlin noise +

randomSize = 100
interpSize = 1000

x = np.linspace(0,randomSize,interpSize)
y = np.linspace(0,randomSize,interpSize)

ospx.seed(4)

interpArray = ospx.noise2array(x,y) 


In [4]:
# 3D sine grid

x = np.linspace(0,200,500)
y = np.linspace(0,200,500)
x, y = np.meshgrid(x, y)
interpArray = (np.sin(x)*np.cos(y))
interpSize = 500


In [5]:
# grid that smoothly transitions between sine and square wave

x = np.linspace(0,200,500)
y = np.linspace(0,200,500)
x, y = np.meshgrid(x, y)
interpArray = ((np.sin(x)*np.cos(y)) * (x/400 + y/400) ) + ((2/np.pi*np.arcsin(np.sin(2*np.pi*x)))*(2/np.pi*np.arccos(np.cos(2*np.pi*y))) * (1 - (x/400 + y/400)))  
interpSize = 500

# Wave-Terrain Code

In [11]:
# wave terrain path generator

# set starting values
iterations = 700
searchRadius = 50
startP = [1,1]
interpArray[startP[1]][startP[1]] = 0

audioBuffer = []
audioBuffer.append(interpArray[startP[0]][startP[1]])
pVisited = [startP]
currentP = startP

# prerender patterns for circle and lines 
circlePattern = circleDraw(searchRadius)

# delete duplicates and shuffle randomly to randomize search bias (only relevant if there are duplicate values in wave terrain)
circlePattern = [*set(circlePattern)]
shuffle(circlePattern)
linePatterns = []

for circleP in circlePattern:
    
    linePatterns.append(lineDraw(0,0,circleP[0],circleP[1],searchRadius))

# generate path
for i in range(iterations):

    searchVal = interpArray[currentP[0]][currentP[1]]
    bestFitValue = 1
    bfValueNum = 0
    nextP = []

    for pointNum, circleP in enumerate(circlePattern):

        xCP = currentP[0] + circleP[0]
        yCP = currentP[1] + circleP[1]

        if((0 <= xCP < interpSize) and (0 <= yCP < interpSize)):
            compVal  = abs(interpArray[xCP][yCP] - searchVal) 

            if(compVal < bestFitValue):
                pAlreadyVisited = False

                for pointVis in pVisited:

                    if(xCP == pointVis[0] and yCP == pointVis[1]):
                        pAlreadyVisited = True

                if(not pAlreadyVisited):
                    bestFitValue = compVal
                    bfValueNum = pointNum                    
                    nextP = [xCP, yCP]         

    pVisited.append(nextP)
    # interpArray[nextP[0]][nextP[1]] = 0 
    fitLPattern = linePatterns[bfValueNum]
    fitLPattern = fitLPattern[1:]

    for patternPoint in fitLPattern:
        audioBuffer.append(interpArray[currentP[0] + patternPoint[0]][currentP[1] + patternPoint[1]])

    currentP = nextP

In [ ]:
# plots wavetable and generative audio-path

pathX, pathY = zip(*pVisited)
plt.plot(pathX, pathY, color='white', linewidth=0.5)
plt.imshow(interpArray)
plt.colorbar()
plt.show()

In [ ]:
# generate audio without gendy modualtion, highly inefficient and doesnt work when wrong sample rate is chosen, for testing purposes only 

from scipy.io.wavfile import write
sampleRate = 44100
stepLength = 200
audio = np.linspace(start=audioBuffer[:-1], stop=audioBuffer[1:], num=round(sampleRate * ((stepLength / searchRadius) / 1000)), endpoint=False, axis=1)
audio = audio.flatten()
scaled = np.int32(audio / np.max(np.abs(audio)) * 2147483647)
write('audio9.wav', sampleRate, scaled.astype(np.int32))
ipd.Audio(scaled, rate=sampleRate)

# Gendy 3

In [12]:
# gendy 3 implementation for x coordinates (based on Hoffmann: Music out of Nothing)

# reversed random distributions as seen in music out of nothing 
def distribute(function, randNum, coeff1, coeff2 = None):
    if function == 'cauchy':
        return coeff1 * np.tan(np.pi * (randNum - 0.5))
    if function == 'logistic':
        return - (np.log((1 - randNum) / randNum) + coeff2) / coeff1
    if function == 'cosine':
        return coeff1 * np.log(np.tan((np.pi * randNum) / 2))
    if function == 'arcsine':
        return coeff1 * (0.5 - (0.5 * np.sin((0.5 - randNum) + np.pi)))            
    if function == 'exponential':
        return - (np.log(1 - randNum) / coeff1)

# mirror point between two boundarys
def mirrorPoint(value, minimum, maximum):
    return (minimum + abs(((0 - value + maximum) % ((minimum - maximum) * 2)) - (minimum - maximum) ))        

# dual random walk
def mirrorPointX(fMin, fMax, nMin, nMax, preValue, randomDistValue):
    return mirrorPoint(mirrorPoint(randomDistValue, fMin, fMax) + preValue, nMin, nMax)

# generate random uniform distribution 
def generateRandomValues(pointNum, function, coeff1, coeff2 = None):
    uniRandomList = np.random.default_rng().uniform(low=0.0, high=1.0, size=pointNum)
    distRandomList = []
    for i in range(pointNum):
        if coeff2 != None:
            distRandomList.append(distribute(function, uniRandomList[i], coeff1, coeff2))
        else:
            distRandomList.append(distribute(function, uniRandomList[i], coeff1))
    return distRandomList    

In [ ]:
# gendy audio generation

from scipy.io.wavfile import write

sampleRate = 44100
segments = 10
fMin = - 10
fMax = 10
nMin = 5
nMax = 600
dist = 'cosine'
coeff1 = 2
coeff2 = None

# start value
xValues = [0]

# array of random uniform numbers
randomNumbers = generateRandomValues(len(audioBuffer), dist, coeff1, coeff2)

for i in range(1, segments):
    xValues.append(mirrorPointX(fMin, fMax, nMin, nMax, 0, randomNumbers[i]))

for i in range(segments, len(audioBuffer)):
    xValues.append(mirrorPointX(fMin, fMax, nMin, nMax, xValues[i - segments], randomNumbers[i]))

for i in range(len(xValues)):
    xValues[i] = xValues[i - 1] + xValues[i]

xAudioLength = np.arange(xValues[-1])
audio = np.interp(xAudioLength, xValues, audioBuffer) 
scaled = np.int32(audio / np.max(np.abs(audio)) * 2147483647)
# write('FinalRenders\gendyPart3.3.wav', sampleRate, scaled.astype(np.int32))
ipd.Audio(scaled, rate=sampleRate)

In [84]:
write('FinalRenders\gendyPart22 .wav', sampleRate, scaled.astype(np.int32))

In [ ]:
plt.figure(figsize=(40, 6))
# xPoints = np.linspace(0,len(audioBuffer[:5000]),len(audioBuffer[:5000]))
plt.plot(xValues[5000:7500], audioBuffer[5000:7500], '-o')
plt.show()

In [119]:
np.savetxt("FinalRenders\gendyPart6WaveTerrain.csv", pVisited, delimiter=",")
np.savetxt("FinalRenders\gendyPart6Time.csv", xValues, delimiter=",")